# How can we improve hyper parameter tuning and visualization?

Comet ML: Comet lets you track code, experiments, and results on ML projects. It’s fast, simple, and free for open source projects.

Why is it useful? Their motivation says it all: "Thousands of experiment results are lost every day. We created Comet to push machine learning research and encourage reproducibility."



## 1. Quickstart
- sign up to their service to get your own API key
- try out the quick start tutorial copied below


In [6]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from comet_ml import Experiment

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

def get_data():
    mnist = input_data.read_data_sets("/tmp/tensorflow/mnist/input_data/", one_hot=True)
    return mnist

def build_model_graph(hyper_params):
    # Create the model
    x = tf.placeholder(tf.float32, [None, 784]) # 28 x 28 pixels, flattened
    W = tf.Variable(tf.zeros([784, 10])) # 10 classes
    b = tf.Variable(tf.zeros([10]))
    y = tf.matmul(x, W) + b

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))
    train_step = tf.train.GradientDescentOptimizer(hyper_params['learning_rate']).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_, axis=1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

    return train_step, cross_entropy, accuracy, x, y, y_

def train(hyper_params):
    mnist = get_data()

    # Get graph definition, tensors and ops
    train_step, cross_entropy, accuracy, x, y, y_ = build_model_graph(hyper_params)

    experiment = Experiment(api_key="yourAPIKEY", project_name="quickstart-project")
    experiment.log_parameters(hyper_params)
    experiment.log_dataset_hash(mnist)
    experiment.log_code()

    with tf.Session() as sess:
        with experiment.train():
            sess.run(tf.global_variables_initializer())
            experiment.set_model_graph(sess.graph)

            for i in range(hyper_params["steps"]):
                batch = mnist.train.next_batch(hyper_params["batch_size"])
                experiment.set_step(i)
                # Compute train accuracy every 10 steps
                if i % 10 == 0:
                    train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
                    print('step %d, training accuracy %g' % (i, train_accuracy))
                    experiment.log_metric("accuracy",train_accuracy,step=i)

                # Update weights (back propagation)
                _, loss_val = sess.run([train_step, cross_entropy],
                                       feed_dict={x: batch[0], y_: batch[1]})

                experiment.log_metric("loss",loss_val,step=i)

        ### Finished Training ###

        with experiment.test():
            # Compute test accuracy
            acc = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})
            experiment.log_metric("accuracy",acc)
            print('test accuracy %g' % acc)



In [9]:
hyper_params = {"learning_rate": 0.01, "steps": 1000, "batch_size": 50}
train(hyper_params)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/lina-weichbrodt/quickstart-project/9a83c14851ed4113b08d72180094688a
COMET INFO:   Metrics:
COMET INFO:      test_accuracy: 0.9079
COMET INFO:     train_accuracy: 0.92
COMET INFO:         train_loss: 0.3648991
COMET INFO: ----------------------------
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/lina-weichbrodt/quickstart-project/c4b52a50cef7487d9beff6c8f0dfc864



step 0, training accuracy 0.18
step 10, training accuracy 0.34
step 20, training accuracy 0.66
step 30, training accuracy 0.66
step 40, training accuracy 0.74
step 50, training accuracy 0.74
step 60, training accuracy 0.7
step 70, training accuracy 0.68
step 80, training accuracy 0.72
step 90, training accuracy 0.72
step 100, training accuracy 0.8
step 110, training accuracy 0.8
step 120, training accuracy 0.86
step 130, training accuracy 0.74
step 140, training accuracy 0.8
step 150, training accuracy 0.84
step 160, training accuracy 0.68
step 170, training accuracy 0.84
step 180, training accuracy 0.78
step 190, training accuracy 0.74
step 200, training accuracy 0.74
step 210, training accuracy 0.88
step 220, training accuracy 0.78
step 230, training accuracy 0.9
step 240, training accuracy 0.76
step 250, training accuracy 0.84
step 260, training accuracy 0.88
step 270, training accuracy 0.84
step 280, training accuracy 0.94
step 290, training accuracy 0.88
step 300, training accurac